In [33]:
from sentence_transformers import SentenceTransformer
from config.common import cfg
import os
from os.path import join
import polars as pl
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

In [34]:
news = pl.read_parquet(join(cfg['data_path'], 'news.parquet'))

In [35]:
train=pl.read_parquet(join(cfg['train_data_path'], 'behaviors.parquet'))
dev=pl.read_parquet(join(cfg['dev_data_path'], 'behaviors.parquet'))

In [36]:
news.columns

['news_id',
 'category',
 'subcategory',
 'title',
 'abstract',
 'url',
 'title_entities',
 'abstract_entities']

In [37]:
userID_mapping = {}
itemID_mapping = {}

for line in train.iter_rows(named=True):
    userID = line['user_id']
    history = line['history']
    for itemID in history:
        if userID not in userID_mapping:
            userID_mapping[userID] = len(userID_mapping) + 1

        if itemID not in itemID_mapping:
            itemID_mapping[itemID] = len(itemID_mapping) + 1
for line in dev.iter_rows(named=True):
    userID = line['user_id']
    history = line['history']
    for itemID in history:
        if userID not in userID_mapping:
            userID_mapping[userID] = len(userID_mapping) + 1

        if itemID not in itemID_mapping:
            itemID_mapping[itemID] = len(itemID_mapping) + 1
for itemID in news.iter_rows(named=True):
    itemID=itemID['news_id']
    if itemID not in itemID_mapping:
            itemID_mapping[itemID] = len(itemID_mapping) + 1

np.save(cfg['user_dict'], userID_mapping)
print("user_num:", len(userID_mapping))
print("the first five userID mapping:", list(userID_mapping.items())[:5])
np.save(cfg['item_dict'], itemID_mapping)
print("item_num:", len(itemID_mapping))
print("the first five itemID mapping:", list(itemID_mapping.items())[:5])


user_num: 94057
the first five userID mapping: [('U13740', 1), ('U91836', 2), ('U73700', 3), ('U34670', 4), ('U8125', 5)]
item_num: 65239
the first five itemID mapping: [('N55189', 1), ('N42782', 2), ('N34694', 3), ('N45794', 4), ('N18445', 5)]


In [38]:
data = pl.read_parquet(os.path.join(cfg['train_data_path'], "behaviors.parquet"))

data_sorted = data.sort(["user_id", "time"])

# 对每个用户取最后一条记录
data = data_sorted.group_by("user_id").tail(1)


def prepare_data(data_dict):
    "数据保存polar形式"
    rows = []
    for userID, item_sequenec in data_dict.items():
        history = item_sequenec[:-1]
        target = item_sequenec[-1]
        rows.append({'user': userID, 'history': history, 'target': target})
    return pl.DataFrame(rows)


"划分训练集，测试集，验证集"
train_data = {}
val_data = {}
test_data = {}

for row in data.iter_rows(named=True):
    userID = row["user_id"]  # 注意列名可能是 'userID' 或 'user_id'
    item_sequence = [itemID_mapping[t] for t in  row["history"]]
    if len(item_sequence) > 2:
        train_data[userID] = item_sequence[:-2]
        val_data[userID] = item_sequence[:-1]
        test_data[userID] = item_sequence

print("training data:", list(train_data.items())[:5])
print("validation data:", list(val_data.items())[:5])
print("testing data:", list(test_data.items())[:5])

train_df = prepare_data(train_data)
print("\nTraining data shape:", train_df.shape)
print("the first 3 rows of training data:\n", train_df.head(3))
test_df = prepare_data(test_data)
print("\nTesting data shape:", test_df.shape)
print("the first 3 rows of testing data:\n", test_df.head(3))
val_df = prepare_data(val_data)
print("\nValidation data shape:", val_df.shape)
print("the first 3 rows of validation data:\n", val_df.head(3))

print("Data saved to parquet files.")
train_df.write_parquet(os.path.join(cfg['train_data_path'], "train_df.parquet"))
test_df.write_parquet(os.path.join(cfg['train_data_path'], "test_df.parquet"))
val_df.write_parquet(os.path.join(cfg['train_data_path'], "valid_df.parquet"))

training data: [('U46', [21, 607, 15040, 4491, 5220, 18465, 685, 2472, 552, 509, 1083, 286, 1003, 4019, 288, 6253, 3099, 1651]), ('U63190', [839, 306, 2119, 939, 11913]), ('U36425', [1032, 1, 21, 2726, 2431, 7, 61, 61, 291]), ('U59192', [1, 669, 2869, 439, 286]), ('U20734', [866, 17088, 997])]
validation data: [('U46', [21, 607, 15040, 4491, 5220, 18465, 685, 2472, 552, 509, 1083, 286, 1003, 4019, 288, 6253, 3099, 1651, 113]), ('U63190', [839, 306, 2119, 939, 11913, 539]), ('U36425', [1032, 1, 21, 2726, 2431, 7, 61, 61, 291, 829]), ('U59192', [1, 669, 2869, 439, 286, 289]), ('U20734', [866, 17088, 997, 2405])]
testing data: [('U46', [21, 607, 15040, 4491, 5220, 18465, 685, 2472, 552, 509, 1083, 286, 1003, 4019, 288, 6253, 3099, 1651, 113, 2843]), ('U63190', [839, 306, 2119, 939, 11913, 539, 1065]), ('U36425', [1032, 1, 21, 2726, 2431, 7, 61, 61, 291, 829, 3728]), ('U59192', [1, 669, 2869, 439, 286, 289, 1511]), ('U20734', [866, 17088, 997, 2405, 1630])]

Training data shape: (46673, 3)

In [39]:
model_path = cfg['emb_model_path']

# 加载本地模型
model = SentenceTransformer(model_path, device='cuda')

Loading weights:   0%|          | 0/99 [00:00<?, ?it/s]

In [40]:
item_embeddings = []
for row in tqdm(news.iter_rows(named=True),total=news.shape[0]):
    itemID = itemID_mapping[row['news_id']]

    semantics = f"Title: {row.get('title','')}\n"

    embedding = model.encode(semantics)
    item_embeddings.append({'ItemID': itemID, 'embedding': embedding.tolist()})

# Convert to DataFrame
item_emb_df = pd.DataFrame(item_embeddings)

print("\nItem embeddings DataFrame shape:", item_emb_df.shape)
print("The first 3 rows of item embeddings DataFrame:\n", item_emb_df.head(3))

# Save to parquet file
item_emb_df.to_parquet(join(cfg['embed_path'],'item_emb_title.parquet'), index=False)

  0%|          | 0/65238 [00:00<?, ?it/s]


Item embeddings DataFrame shape: (65238, 2)
The first 3 rows of item embeddings DataFrame:
    ItemID                                          embedding
0   12963  [-0.0234527587890625, -0.0164642333984375, 0.0...
1   11453  [-0.01428985595703125, -0.0253753662109375, 0....
2    9287  [0.008453369140625, -0.0794677734375, 0.013267...
